# Amplify Diff: Add difference with doubled M #

Making "amplified difference" merge, an intermediate step for making the Re-Basin merge. 

## Abstract ##

- This is a special case of my *mega* merge. Usually `add_diff`  is not supassing 1, at least in [A1111](https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Features#checkpoint-merger). This time I need 2.

## Expected "Model A" and "Model B" ##

- Both models are expected to be merged from [sd-mecha](https://github.com/ljleb/sd-mecha), **and comed from the same model pool.** Currently my model pool consists of of 102 models.
- Base Model (not used): OG SDXL 1.0
- Model A: $A$ as [Uniform merge](https://arxiv.org/abs/2203.05482).
- Model B: $B$ as [DARE (ICML2024)](https://arxiv.org/abs/2311.03099), "TIES-SOUP w/ DARE", along with DROP only (no rescale)

## Expected output and algorithm discussion ##

- Target model: $A + (B - A) * 2 = 2B - A$ where $M=2$
- $M=1$ will definitely return $B$.
- Model B, which is TIES based, uses "averaging with sign (or movement) voting", which should be close to averaging also.
- Target model will be merged again with Model A under [Git Basin](https://arxiv.org/abs/2209.04836), which pivots on the [mid point](https://en.wikipedia.org/wiki/Midpoint) first: $(A + 2B - A) / 2 = B$,which is between $A$ and $2B - A$, the *amplified differnece*. 
- Then under an [optimization problem](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linear_sum_assignment.html), the model will shift slightly from B. 
- The *slight shift* has been discovered in [my previous work](../ch01/rebasin.md#yes-time-to-make-pr), and I will do it [seperately](https://github.com/ogkalu2/Merge-Stable-Diffusion-models-without-distortion).

## Required libraries ##

- `torch>=2.0.1`
- `tensordict`
- `sd-mecha` (I prefer [clone](https://github.com/6DammK9/sd-mecha/tree/main) the source code inplace,current version as on 241011, branch [della](https://github.com/6DammK9/sd-mecha/tree/della)), until [this PR](https://github.com/ljleb/sd-mecha/pull/41) has been merged.
- [safetensors](https://huggingface.co/docs/safetensors/index)
- [diffusers](https://huggingface.co/docs/diffusers/installation)
- [pytorch](https://pytorch.org/get-started/locally/#windows-python)

## Limitation ##

- ~~VAE remains unmanaged.~~ VAE can be picked from one of the raw models.
- SDXL models only. I don't need this for SD1 and SD2.
- Safetensors only. 

## WTF why and will it work? ##

- Yes. [It is part of my research](./README_XL.md).
- Image comparasion will be listed there.

## Importing libraries ##

In [1]:
# Built-in
import time
import os
import math

# Is dependency fufilled?
import torch

from tqdm import tqdm

In [2]:
torch.__version__

'2.6.0+cu126'

In [3]:
# Import the main module.
import sd_mecha

sd_mecha.set_log_level() #INFO

In [4]:
# Fix for OMP: Error #15
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

## User input session starts here. ##

Specify all the paths.

In [5]:
DIR_BASE = "F:/NOVELAI/astolfo_mix/sdxl/" #To set up merger

DIR_RAW = "raw/" #To load N models
DIR_CLIP = "clip/"  #To write 3N models
DIR_UNET = "unet/" #To write N models
DIR_FINAL = "./" #To write 1 model

Quick check on directory and make the model name prefix.

In [6]:
MECHA_RECIPE_EXT = ".mecha"
MECHA_MODEL_EXT = ".safetensors"

Define model A and B.

In [7]:
MODEL_A = "noobaiXLNAIXL_epsilonPred11Version"
MODEL_B = "x6a-AstolfoKarMix-25071602-5818193"

MODEL_A_PATH = "{}{}{}".format(DIR_FINAL, MODEL_A, MECHA_MODEL_EXT)
MODEL_B_PATH = "{}{}{}".format(DIR_FINAL, MODEL_B, MECHA_MODEL_EXT)

In [8]:
print("Model A: {}".format(MODEL_A_PATH))
print("Model B: {}".format(MODEL_B_PATH))

Model A: ./noobaiXLNAIXL_epsilonPred11Version.safetensors
Model B: ./x6a-AstolfoKarMix-25071602-5818193.safetensors


Insert version number, and the... *"AstolfoMix"*.

If you want to make multiple versions of AstolfoMix with different algorithms, I suggest modify the `MODEL_NAME_KEYWORD`.

In [9]:
MODEL_NAME_SUFFIX = "25071602-5818193" #yymmddxx-commit
MODEL_NAME_KEYWORD = "AstolfoKarmix"

Change if your PC is in trouble.

My WS: [Xeon 8358 ES, X12DPI-N6, 512GB DDR4, 2x RTX2080ti 22G, P4510 4TB *2](https://github.com/6DammK9/nai-anime-pure-negative-prompt/blob/main/ch04/ice_lake_ws.md). Overkill for a merger.

In [10]:
g_seed = 250716 #For reproducible result
g_device = "cpu" #"cuda:0" if True else "cpu"  #I have 2 GPUS and this is the CPU slot
g_precision_while_merge = torch.float64 if "cuda" in g_device else torch.float64 #I have RAM
g_precision_final_model = torch.float16 if "cuda" in g_device else torch.float16 #fp16

#240407: 2**34 will throw NaN issue. Stay with default = 2**28
g_total_buffer_size=2**28
#240507: (Not effective) DARE requires single thread to prevent OOM
g_threads = 1 #if False else None

## User input shuold ends here. ##

Define output model name. I want to keep the format, however I need to manage the name manually.

In [11]:
MODEL_NAME_AMPLIFIED = "amp-{}-{}".format(MODEL_NAME_KEYWORD,MODEL_NAME_SUFFIX) #amp-AstolfoMix-240507-4edc67c

In [12]:
print("Amplified model:                         {}".format(MODEL_NAME_AMPLIFIED))

Amplified model:                         amp-AstolfoKarmix-25071602-5818193


Define recipe extension, and make the model output path (Note that it is still being formatted)

In [13]:

OS_MODEL_PATH_AMPLIFIED = "{}{}{}".format(DIR_BASE,MODEL_NAME_AMPLIFIED,MECHA_MODEL_EXT)
RECIPE_PATH_AMPLIFIED = "{}{}{}".format(DIR_BASE,MODEL_NAME_AMPLIFIED,MECHA_RECIPE_EXT)
MECHA_MODEL_PATH_AMPLIFIED = "{}{}{}".format(DIR_FINAL,MODEL_NAME_AMPLIFIED,MECHA_MODEL_EXT)

In [14]:
print("Amplified model path    (OS): {}".format(OS_MODEL_PATH_AMPLIFIED))
print("Amplified model recipe  (OS): {}".format(RECIPE_PATH_AMPLIFIED))
print("Amplified model path (mecha): {}".format(MECHA_MODEL_PATH_AMPLIFIED))

Amplified model path    (OS): F:/NOVELAI/astolfo_mix/sdxl/amp-AstolfoKarmix-25071602-5818193.safetensors
Amplified model recipe  (OS): F:/NOVELAI/astolfo_mix/sdxl/amp-AstolfoKarmix-25071602-5818193.mecha
Amplified model path (mecha): ./amp-AstolfoKarmix-25071602-5818193.safetensors


### Right before the merging stuffs, I need to clear some hardcode. ###
- Note that I assume the VAE has been fixed.

In [15]:
MECHA_IS_SDXL = "sdxl-sgm"
AMP_FACTOR = 2.0
DISABLE_CLIP = False

In [16]:
# 0.0.26: sd_mecha.RecipeMerger
# 1.0.4: Just config
MERGE_CONFIG = {
    "model_dirs": [DIR_BASE],
    "merge_device": g_device,
    "merge_dtype": g_precision_while_merge,
    "output_device": g_device,
    "output_dtype": g_precision_final_model,
    "total_buffer_size": g_total_buffer_size,
    "omit_extra_keys": True,
    "threads": g_threads
}

### Amplified Difference ###

- Notice that `c` is required and it is same as `model_a`
- Note that `alpha` is applied globally so it is just a single number.
- Somehow `clip_to_ab` has been implemented to "prevent model weight out of range", so I need to turn it off.

In [17]:
def make_recipe_amp_diff():
    casted_model_a = sd_mecha.model(MODEL_A_PATH, MECHA_IS_SDXL)
    casted_model_b = sd_mecha.model(MODEL_B_PATH, MECHA_IS_SDXL)
    casted_recipe = sd_mecha.add_difference(casted_model_a, casted_model_b, casted_model_a, alpha=AMP_FACTOR, clamp_to_ab=DISABLE_CLIP)
    return casted_recipe

In [18]:
recipe_amp_diff = make_recipe_amp_diff()

In [19]:
sd_mecha.serialize(recipe_amp_diff, output=RECIPE_PATH_AMPLIFIED)

INFO: Saving recipe to F:\NOVELAI\astolfo_mix\sdxl\amp-AstolfoKarmix-25071602-5818193.mecha


'version 0.1.0\nmodel "noobaiXLNAIXL_epsilonPred11Version.safetensors" model_config="sdxl-sgm" merge_space="weight"\nmodel "x6a-AstolfoKarMix-25071602-5818193.safetensors" model_config="sdxl-sgm" merge_space="weight"\nmerge "subtract" &1 &0\nmerge "add_difference" &0 &2 2.0\nmerge "fallback" &3 &0'

## Time for action ##

In [20]:
ts = time.time()

### Amplified Difference ###

- No `fallback_model` because I expect model A and B are well prepared.

In [21]:
tss = time.time()

if os.path.isfile(OS_MODEL_PATH_AMPLIFIED):
    print("Merged model is present. Skipping.")
else:
    #scheduler.merge_and_save(recipe_amp_diff, output=MECHA_MODEL_PATH_AMPLIFIED, save_dtype=g_precision_final_model, total_buffer_size=g_total_buffer_size, threads=g_threads)
    sd_mecha.merge(recipe_amp_diff, output=MECHA_MODEL_PATH_AMPLIFIED, **MERGE_CONFIG)

tse = time.time()
print("Merge time: {} sec".format(int(tse - tss)))    

Merging recipe: 100%|██████████| 2519/2519 [00:52<00:00, 47.88it/s, key=ztsnr, shape=[0]]                                                                                           

Merge time: 52 sec


Full operation time.

In [22]:
te = time.time()
print("Total time: {} sec".format(int(te - ts)))

Total time: 52 sec


Finally move the models to the [git rebasin](https://github.com/ogkalu2/Merge-Stable-Diffusion-models-without-distortion) folder and merge them.

```sh
python SD_rebasin_merge.py --model_a x101a-AstolfoMix-24051501-29bac1a.safetensors --model_b amp-AstolfoMix-24051501-29bac1a.safetensors
```